Data Collection and Data Preparation

In [1]:
import openai

In [3]:
import os
from dotenv import load_dotenv
import PyPDF2

from sentence_transformers import SentenceTransformer
import faiss

import streamlit as st

SyntaxError: invalid syntax (1829268383.py, line 1)

In [55]:
# Load environment variables from the .env file
load_dotenv()

# Access the OpenAI API key
openai_api_key = os.getenv("OPENAI_API_KEY")

print(openai_api_key)

sk-xGZoo8wC4Y2cwJjKUExwUUY0cgD8e572StmHJOW_DJT3BlbkFJy4nN2oTfGVJPjkxgv5z0ImGLIjJzFlOBHDhaLfy2oA


In [62]:
import openai

ImportError: cannot import name 'Sequence' from 'typing_extensions' (c:\users\lenovo\appdata\local\programs\python\python38\lib\site-packages\typing_extensions.py)

In [38]:
def extract_text_from_pdfs(pdf_dir):
    """
    Extract text from all PDF files in the specified directory.
    
    Args:
        pdf_dir (str): Directory containing PDF files.
        
    Returns:
        List of strings: Extracted text from each PDF file.
    """
    text_data = []
    for pdf_file in os.listdir(pdf_dir):
        if pdf_file.endswith('.pdf'):
            with open(os.path.join(pdf_dir, pdf_file), 'rb') as file:
                reader = PyPDF2.PdfReader(file)
                text = ''
                for page in range(len(reader.pages)):  # Iterate over pages
                    text += reader.pages[page].extract_text()  # Extract text from each page
                text_data.append(text)
                print(f"Extracted from {pdf_file}: {text}")  # Print text from each file
    return text_data

# Directory containing PDF files
pdf_texts = extract_text_from_pdfs('data')  # Store the return value
print(pdf_texts)  # Print the final output

Extracted from Automated Road Extraction From High Resolution Satellite Images.pdf:  Procedia Technology   24  ( 2016 )  1460 – 1467 Available online at www.sciencedirect.com
ScienceDirect
2212-0173 © 2016 The Authors. Published by Elsevier Ltd. This is an open access article under the CC BY-NC-ND license 
(http://creativecommons.org/licenses/by-nc-nd/4.0/).
Peer
-review under responsibility of the organizing committee of ICETEST – 2015
doi: 10.1016/j.protcy.2016.05.180 
International Conference on Emerging Trends in Engineering, Science and Technology 
(ICETEST - 2015) 
Automated Road Extraction  From High Resolution Satellite Images  
Jose Hormesea,*, Dr. C. Saravananb   
aResearch  Scholar, Dept  of Computer Centre , NIT, Durgapur ,713209,  India  
bResearch Guide, Dept of Computer  Centre,  NIT, Durgapur,713209,  India  
Abstract  
The importance of road extraction from satellite images arises from the fact that it greatly enhances the efficiency of map 
generation and thus can be 

In [39]:
# Building the knowledge base

# Load pre-trained sentence transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Encode the extracted text data
embeddings = model.encode(pdf_texts)

# Initialize FAISS index
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

# Save the FAISS index to disk
faiss.write_index(index, 'faiss_index.idx')

In [42]:
class RetrievalAgent:
    def __init__(self, index, model):
        """
        Initialize the Retrieval Agent.
        
        Args:
            index: FAISS index for document retrieval.
            model: Sentence transformer model for encoding queries.
        """
        self.index = index
        self.model = model

    def retrieve(self, query, top_k=5):
        """
        Retrieve top-k relevant documents for a given query.
        
        Args:
            query (str): User query.
            top_k (int): Number of top documents to retrieve.
            
        Returns:
            List of indices of the top-k relevant documents.
        """
        query_embedding = self.model.encode([query])
        distances, indices = self.index.search(query_embedding, top_k)
        return indices

class GenerationAgent:
    def __init__(self, openai_api_key):
        """
        Initialize the Generation Agent.
        
        Args:
            openai_api_key (str): API key for OpenAI.
        """
        self.openai_api_key = openai_api_key

    def generate(self, prompt):
        """
        Generate a response based on the given prompt using OpenAI.
        
        Args:
            prompt (str): Input prompt for generation.
            
        Returns:
            str: Generated response.
        """
        openai.api_key = self.openai_api_key
        response = openai.Completion.create(
            engine="davinci-codex",
            prompt=prompt,
            max_tokens=150
        )
        return response.choices[0].text.strip()

# Initialize agents
retrieval_agent = RetrievalAgent(faiss.read_index('faiss_index.idx'), model)
generation_agent = GenerationAgent('OPENAI_API_KEY')

In [41]:
def answer_query(query):
    """
    Answer a user query by retrieving relevant documents and generating a response.
    
    Args:
        query (str): User query.
        
    Returns:
        str: Generated answer.
    """
    # Retrieve relevant documents
    doc_indices = retrieval_agent.retrieve(query)
    relevant_docs = [pdf_texts[i] for i in doc_indices]

    # Combine documents into a single prompt
    combined_docs = "\n\n".join(relevant_docs)
    prompt = f"Answer the following query based on these documents:\n\n{query}\n\n{combined_docs}"

    # Generate answer
    answer = generation_agent.generate(prompt)
    return answer

# Test the system with a sample query
query = "What are the latest advancements in AI?"
response = answer_query(query)
print(response)

TypeError: only integer scalar arrays can be converted to a scalar index

In [ ]:
# Streamlit app title
st.title("AI Research Assistant")

# Input field for user query
query = st.text_input("Enter your research query:")

# Display the generated response when a query is entered
if query:
    response = answer_query(query)
    st.write(response)